# Billion-scale random spanning tree with 🍇🍇 GRAPE 🍇🍇
In this tutorial, I will show you how to use the [GRAPE library](https://github.com/AnacletoLAB/grape) to compute a graph's random spanning tree is a tree structure that spans the entire graph and represents the relationships between nodes. These algorithms are designed for unweighted undirected graphs and produce random spanning trees without the possibility of setting a seed. This task is common in many applications and a standard step in graph analysis.

I will then briefly explain what parallel work-stealing algorithms and some brief details on the random spanning tree algorithm available in [GRAPE!](https://github.com/AnacletoLAB/grape) are.

By the end of the tutorial, you will have a good understanding of how to use [GRAPE!](https://github.com/AnacletoLAB/grape) to compute a random spanning tree of a graph and apply this knowledge to your projects.

[Remember to ⭐ GRAPE!](https://github.com/AnacletoLAB/grape)

### What is GRAPE?
[🍇🍇 GRAPE 🍇🍇](https://github.com/AnacletoLAB/grape) is a graph processing and embedding library that enables users to easily manipulate and analyze graphs. With [GRAPE](https://github.com/AnacletoLAB/grape), users can efficiently load and preprocess graphs, generate random walks, and apply various node and edge embedding models. Additionally, [GRAPE](https://github.com/AnacletoLAB/grape) provides a fair and reproducible evaluation pipeline for comparing different graph embedding and graph-based prediction methods.

*The methods shown in the tutorial are available from the nightly version of 🍇 on GitHub, which we'll release on PyPI next week. (Today is 23/12/2022)*

## What is a random spanning tree?
A random [spanning tree of a graph](https://it.wikipedia.org/wiki/Spanning_tree_(networking)) is a tree structure that spans the entire graph and includes all of the nodes in the graph. It is called "random" because it is constructed using a stochastic process rather than following a predetermined order or rule. In this case, the randomness derives from the tree being built in parallel by multiple threads that operate in a work-stealing manner.

<img src="https://github.com/AnacletoLAB/grape/blob/main/images/random_spanning_tree.png?raw=true" width=500 />

### What are their uses?
Random spanning trees have several uses in the analysis and understanding of large, complex graphs.

#### Robustness of a graph
One use is to evaluate the robustness of the graph. For example, suppose a graph represents a network of roads or a power grid. In that case, it is crucial to ensure that it remains connected even if some of its edges fail. As we will see, building a random spanning tree is extremely fast. By constructing multiple random spanning trees of the graph and analyzing their structure, it is possible to determine the likelihood that the graph will remain connected under different scenarios.

#### Graph analysis and clustering heuristics
Another use of random spanning trees is to identify patterns or trends in the graph that may not be immediately apparent. For example, suppose a graph represents a social network. A random spanning tree can identify clusters of individuals more closely connected to each other than the rest of the network. This can be useful for understanding the network's structure and identifying influential nodes or communities within it.

#### A building block in graph partitioning and node embedding
Random spanning trees can also be used as a building block in other algorithms, such as graph partitioning and node embedding. Graph partitioning involves dividing a graph into smaller, more manageable subgraphs while preserving the structure and properties of the original graph. Random spanning trees can identify clusters or communities within the graph, which can then be used to divide the graph into smaller subgraphs. Node embedding involves representing the nodes in a graph in a low-dimensional vector space to facilitate ML tasks such as classification or clustering. Random spanning trees can create node embeddings by constructing a tree for each node and using the tree's structure to encode information about the node and its relationships within the graph.

Overall, spanning trees can be a valuable tool for understanding and analyzing graphs and have a wide range of applications in fields such as machine learning and data science.

### Representing a spanning tree: predecessors vector
In the context of data structures used to represent spanning trees, a predecessor vector is a vector that stores, for each node in the tree, the node's predecessor in the tree. The predecessor of a node is the node that immediately precedes it in the tree structure. It is usually determined by the order in which the nodes were visited during the tree construction.

For example, consider the following tree:

<img src="https://github.com/AnacletoLAB/grape/blob/main/images/tree_example.png?raw=true" width=150 />

In this tree, node `0` is the root, and nodes `1` and `2` are its children. Node `3` is a child of node `1`, and node `4` is a child of node `2`. If we were to construct a predecessors vector for this tree, it would look like this:

```python
predecessors = [0, 0, 0, 1, 2]
```

The first element of the vector represents the root node (node `0`), which has no predecessor, indicated with a self-loop (the predecessor of the node is the node itself). The second element represents node `1`, with node `0` as its predecessor. The third element represents node `2`, with node `0` as its predecessor. The fourth element represents node `3`, which has node `1` as its predecessor, and the fifth element represents node `4`, which has node `2` as its predecessor.

The predecessor vector can efficiently navigate the tree structure and perform operations such as finding the path between two nodes or traversing the tree in a specific order.

## A parallel work-stealing algorithm for random spanning trees

The parallel work-stealing algorithm available from GRAPE which we are going to use was implemented by [Luca Cappelletti](https://www.linkedin.com/in/luca-cappelletti-364a25119/) and [Tommaso Fontana](https://www.linkedin.com/in/tommaso-fontana/) and was described the work by [David Bader](https://www.linkedin.com/in/dbader13/) and [Goujing Cong](https://www.linkedin.com/in/guojing-cong-8691184/) in ["A Fast, Parallel Spanning Tree Algorithm for Symmetric
Multiprocessors (SMPs)"](https://smartech.gatech.edu/bitstream/handle/1853/14355/GT-CSE-06-01.pdf).

This algorithm can execute graphs without significant synchronization steps with regular and irregular topologies. On massive graphs, it has a runtime that decreases linearly with the number of processors. By building on top of [Bader and Cong's algorithm](https://smartech.gatech.edu/bitstream/handle/1853/14355/GT-CSE-06-01.pdf), one can indeed identify and implement many other derivative use cases that may benefit from this work-stealing approach.

This algorithm is efficient because of the implementation details, and detailing this algorithm in a pseudocode would only be detrimental to the understanding of how such a class of algorithms works. I suggest you proceed to read the source code, [which is available here](https://github.com/AnacletoLAB/ensmallen/blob/a46567383441b519fb3f0d2bfe92d4e811974d89/graph/src/trees.rs#L424).

### Parallel algorithms
A [parallel algorithm](https://en.wikipedia.org/wiki/Parallel_algorithm) is designed to be executed concurrently on multiple processors or computing devices to solve a problem faster than a sequential algorithm. Parallel algorithms are often used to solve problems that are too large or complex to be efficiently solved by a single processor. They rely on the fact that modern computers often have multiple processors or cores that can be used to execute tasks concurrently.

There are many parallel algorithms, including divide-and-conquer algorithms, which divide the problem into smaller subproblems that can be solved independently. Different types of parallelism exist, such as task parallelism, which involves executing various tasks concurrently, or data parallelism, which consists of operating on multiple data items together. Both types of parallelism may be used at once in an algorithm.

To design and implement a parallel algorithm, it is often necessary to consider factors such as the number of threads available, the communication and **synchronization** requirements, and the **overhead associated with dividing the problem into smaller pieces and coordinating the execution of the algorithm on multiple processors**.

### Work-stealing algorithms
A [work-stealing algorithm](https://en.wikipedia.org/wiki/Work_stealing) is a parallel programming technique used to dynamically balance the workload among multiple processors or threads in a computer system. It works by having each processor or thread maintain a queue of tasks that need to be completed. If a processor runs out of jobs to work on, it can "steal" tasks from the queue of another processor that still has tasks remaining.

The basic idea behind work-stealing is to allow processors to work independently on their own tasks as much as possible but to also provide a mechanism for redistributing work when one processor becomes idle while others are still busy. This can improve the system's overall performance by ensuring that all processors are kept busy and that the workload is evenly distributed.

Work-stealing algorithms can be beneficial when the workload is not evenly distributed among the processors or when the work can be decomposed into smaller tasks that can be executed independently.

**In graphs, the work relative to processing a high-degree node is often much more intensive than one for processing a low-degree node.**

## Installing GRAPE
First, we install the GRAPE library from PyPI:

In [1]:
!pip install grape -qU

## Experiments
Welcome to the experiments section of this tutorial! In this section, we will put our knowledge into practice by applying the work-stealing parallel random spanning tree algorithm on four different graphs: the [KGCOVID19 knowledge graph](https://www.cell.com/patterns/fulltext/S2666-3899(20)30203-8?_returnURL=https%3A%2F%2Flinkinghub.elsevier.com%2Fretrieve%2Fpii%2FS2666389920302038%3Fshowall%3Dtrue), the [Friendster graph](https://networkrepository.com/friendster.php), the [ClueWeb09 web graph](https://networkrepository.com/web-ClueWeb09.php), and [the WikiData graph](https://www.wikidata.org/wiki/Wikidata:Main_Page).

We run these experiments on a machine with 24 threads and 12 cores.

**Do note that, for the limits of memory of my desktop, I will restart the jupyter after running the experiment on each of the large graphs.**

In my machine I only have 24 threads. You can estimate the expected computation time by interpolating the time estimates on 24 threads and the amount you have:

In [2]:
import os

os.cpu_count()

24

Also, this machine has about `128GB` of RAM:

In [3]:
import psutil
    
psutil.virtual_memory().total / 1024**3 # total physical memory in Bytes

125.7713851928711

### KGCOVID19
We kick off our experiments with a relatively small graph, considering the sizes of the networks we will tackle by the end of it: KGCOVID19, with `574K` nodes and `18M` edges.

#### What is KGCOVID19?
[KGCOVID19](https://doi.org/10.1016%2Fj.patter.2020.100155) is a framework for producing knowledge graphs (KGs) that integrate and integrate biomedical data related to the COVID-19 pandemic. The framework is designed to be flexible and customizable, allowing researchers to create KGs for different downstream applications, including machine learning tasks, hypothesis-based querying, and browsable user interfaces for exploring and discovering relationships in COVID-19 data. The goal of KGCOVID19 is to provide an up-to-date, integrated source of data on SARS-CoV-2 and related viruses, including SARS-CoV and MERS-CoV, to support the biomedical research community in its efforts to respond to the COVID-19 pandemic. The framework can also be applied to other situations where siloed biomedical data must be quickly integrated for various research purposes, including future pandemics.

In [4]:
%%time
from grape.datasets.kghub import KGCOVID19

kgcovid19 = KGCOVID19()

CPU times: user 30.6 s, sys: 3.55 s, total: 34.1 s
Wall time: 11.3 s


We display the number of nodes, `574K` and of undirected edges `18M`.

In [5]:
kgcovid19.get_number_of_nodes(), kgcovid19.get_number_of_edges()

(574232, 18251238)

And now we compute a random spanning tree on KGCOVID19. It should be pretty much instantenous.

In [6]:
%%time
predecessors = kgcovid19.get_random_spanning_tree()

CPU times: user 2.57 s, sys: 598 ms, total: 3.16 s
Wall time: 142 ms


And here is the vector of predecessors representing this random spanning tree:

In [7]:
predecessors

array([     0,      0,      3, ..., 572744,  37278, 573491], dtype=uint32)

### Friendster
[Friendster](https://en.wikipedia.org/wiki/Friendster) was a social networking service launched in 2002. It was one of the first social networking sites and was popular in the early 2000s. The site allowed users to connect with friends and meet new people through the use of personal profiles and networks of friends. Friendster was initially successful but eventually faced competition from more recent social networking sites such as MySpace and Facebook. In 2011, the company announced that it was transitioning from a social networking site to a social gaming site, and in 2015 it was acquired by a Malaysian company.

#### What is the network repository?
[Network Repository](https://networkrepository.com/index.php) is a scientific network data repository that provides interactive visualization and mining tools for analyzing and exploring network data. It is the first interactive repository of its kind. Network Repository is intended to facilitate scientific research on networks by making it easier for researchers to access and analyze an extensive network data collection. It is a valuable resource for researchers in various fields, including network science, bioinformatics, machine learning, data mining, physics, and social science.

#### ⚠️⚠️⚠️ WARNING: Make sure you have enough disk space! ⚠️⚠️⚠️
*Please be aware that this graph is not small and requires a significant amount of disk space to store and work with. Before proceeding with the tutorial, ensure you have enough free space on your hard drive or other storage devices to accommodate the size of the graph. If you do not have sufficient space, you may encounter errors or other issues when downloading or working with the graph. It is important to ensure that you have enough space available before proceeding. If necessary, consider freeing up additional space on your device to make room for the graph.*

In [8]:
!du -sh /bfd/graphs/networkrepository/SocFriendster

97G	/bfd/graphs/networkrepository/SocFriendster


In the next cell we retrieve and load the Friendster dataset from GRAPE, dataset from the [network repository](https://networkrepository.com/index.php).. Do note that we are configuring it to not load the node names and edge types in order to conserve memory. The cell also includes a directive to measure and display the execution time of the code.

In [9]:
%%time
from grape.datasets.networkrepository import SocFriendster

friendster = SocFriendster(
    # We cannot load the node names, as the would require too much memory
    # for my poor old desktop.
    load_nodes=False,
)

CPU times: user 39min 25s, sys: 1min 24s, total: 40min 49s
Wall time: 5min 38s


We display the number of nodes, `65.6M`, and of undirected edges, `1.8G`.

In [10]:
friendster.get_number_of_nodes(), friendster.get_number_of_edges()

(65608366, 1806067135)

And now we compute a random spanning tree on Friendster. In this graph, the work-stealing scales in a fantastic manner and it completes in no time at all.

In [11]:
%%time
predecessors = friendster.get_random_spanning_tree()

CPU times: user 7min 25s, sys: 49.8 s, total: 8min 15s
Wall time: 21.5 s


And here is the vector of predecessors representing this random spanning tree:

In [12]:
predecessors

array([       0,        0,      222, ..., 63090471, 63090471, 65608362],
      dtype=uint32)

### ClueWeb
[The ClueWeb09 dataset](http://lemurproject.org/clueweb09/) was created to support research on information retrieval and related human language technologies; it consists of about `1.7` billion web pages that were collected in January and February 2009 and the roughly `8` billion undirected links.

It is used for research on information retrieval and related human language technologies and is used by several tracks of the TREC conference. The dataset includes web pages in various languages and a web graph that includes unique URLs and total outlinks for the entire dataset and for a subset called TREC Category B (the first 50 million English pages). The ClueWeb09 dataset and subsets are distributed in different formats, including as tarred/gzipped files on hard disk drives and as a subset that is downloaded from the web. The Lemur Project provides online services for searching and interacting with the ClueWeb09 dataset, including an Indri search engine for searching the English and Japanese subsets and Wikipedia, as well as a batch query service and an attribute lookup service. The Lemur Project also offers hosted copies of the ClueWeb09 dataset for organizations that have licenses to use it.

*We also retrieve this graph from [Network Repository](https://networkrepository.com/index.php)*

#### ⚠️⚠️⚠️ This is a big graph! Make sure you have the disk space! ⚠️⚠️⚠️
*This is a warning to ensure that users have sufficient disk space before downloading and using a large graph. It is important to ensure that you have enough space on your hard drive or another storage device to accommodate the graph size, as attempting to download or work with a graph that is too large for your available space can lead to errors and other issues. It is advisable to check your available disk space before downloading or working with a large graph and free up additional space if necessary.*

In [8]:
!du -sh /bfd/graphs/networkrepository/WebClueweb09/

631G	/bfd/graphs/networkrepository/WebClueweb09/


In the following cell we retrieve and load the `Clueweb09` dataset from the [network repository](https://networkrepository.com/index.php). We configure it to not load the node names in order to conserve memory. The cell also includes a directive to measure and display the execution time of the code.

In [1]:
%%time
from grape.datasets.networkrepository import WebClueweb09

clueweb = WebClueweb09(
    # We cannot load the node names, as the would require too much memory
    # for my poor old desktop.
    load_nodes=False,
)

CPU times: user 2h 48min 28s, sys: 9min 1s, total: 2h 57min 30s
Wall time: 37min 58s


We display the number of nodes, `1.68G`, and of undirected edges, `7.8G`.

In [2]:
clueweb.get_number_of_nodes(), clueweb.get_number_of_edges()

(1684868322, 7811385827)

And now we compute a random spanning tree. In this particular graph, even though it is colossal, the algorithm is able to distribute the load efficiently across the 24 threads and complete in about 6 minutes with minimal synchronization steps required.

In [3]:
%%time
predecessors = clueweb.get_random_spanning_tree()

CPU times: user 2h 6min, sys: 11min 38s, total: 2h 17min 38s
Wall time: 6min 2s


And here is the vector of predecessors representing this random spanning tree:

In [4]:
predecessors

array([         0,          0,     155482, ...,  976219193,  817883058,
       1684868320], dtype=uint32)

## WikiData
[WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page) is a collaborative, multilingual, free knowledge base that can be read and edited by humans and machines. It provides structured data representing the relationships between concepts and entities, including real-world objects, events, ideas and abstract concepts. The data in WikiData is organized into a graph structure, with nodes representing concepts or entities and edges representing relationships between them. For example, a node for the idea "dog" might be connected to other nodes representing specific dog breeds, such as "Labrador Retriever" or "Poodle," through edges that define the relationship "breed of."

The WikiData graph is constantly growing and changing as users contribute new data and edit existing data. It is based on a flexible data model that allows for creation of new properties and classes to represent the relationships between concepts and entities. The WikiData graph is used in various applications, including data integration, natural language processing, and machine learning. It also provides structured data for Wikipedia and other Wikimedia projects.

#### ⚠️⚠️⚠️ This is a big graph! Make sure you have the disk space! ⚠️⚠️⚠️
*This is a warning to ensure that users have sufficient disk space before downloading and using a large graph. It is important to ensure that you have enough space on your hard drive or another storage device to accommodate the graph size, as attempting to download or work with a graph that is too large for your available space can lead to errors and other issues. It is advisable to check your available disk space before downloading or working with a large graph and free up additional space if necessary.*

In [1]:
!du -sh /bfd/graphs/wikidata/WikiData

1,7T	/bfd/graphs/wikidata/WikiData


In the next cell we retrieve and load the WikiData dataset from GRAPE, directly from [WikiData's website](https://www.wikidata.org/wiki/Wikidata:Main_Page). Do note that we are configuring it to not load the node names and edge types in order to conserve memory. The cell also includes a directive to measure and display the execution time of the code.

In [2]:
%%time
from grape.datasets.wikidata import WikiData

wikidata = WikiData(
    # We cannot load the node names, as the would require too much memory
    # for my poor old desktop.
    load_nodes=False,
    # Same thing for the edge types.
    load_edge_types=False
)

CPU times: user 1h 47min 22s, sys: 5min 3s, total: 1h 52min 25s
Wall time: 20min 23s


We display the number of nodes, `1.29G` and of undirected edges `5G`.

In [3]:
wikidata.get_number_of_nodes(), wikidata.get_number_of_edges()

(1294126247, 5040170396)

And now we compute a random spanning tree on WikiData. In this graph, even though it is also colossal, the algorithm is able to distribute the load efficiently across the 24 threads and complete in about four minutes with minimal synchronization steps required.

In [4]:
%%time
predecessors = wikidata.get_random_spanning_tree()

CPU times: user 1h 25min 57s, sys: 7min 13s, total: 1h 33min 10s
Wall time: 4min 4s


And here is the vector of predecessors representing this random spanning tree:

In [5]:
predecessors

array([         0,          0,          0, ..., 1294126241, 1294126242,
       1294126242], dtype=uint32)

## Conclusions

In this tutorial, we learned how to use the [GRAPE](https://github.com/AnacletoLAB/grape) library to compute random spanning trees of large graphs. We discussed the various uses of random spanning trees, including evaluating the robustness of a graph, identifying patterns or trends in the graph, and serving as a building block for other algorithms, such as graph partitioning and node embedding. We also briefly mentioned the concept of parallel work-stealing algorithms and how they can be used to efficiently construct random spanning trees.

I hope you now understand the applications and benefits of random spanning trees and how to use [GRAPE](https://github.com/AnacletoLAB/grape) to compute them for your projects. Do feel free to reach out with any questions or feedback, as I always look for ways to improve this tutorial.

[And remember to ⭐ GRAPE!](https://github.com/AnacletoLAB/grape)